In [7]:
from multiprocessing import Process, Value, Array
import time

def add100(number):
    for i in range(100):
        time.sleep(0.01)
        number.value += 1

if __name__ == "__main__":
    shared_number = Value('i', 0) # two arguments, data type (i for integer) and starting value 
    print("Number at the beginning is", shared_number.value)
    
    p1 = Process(target=add100, args=(shared_number,))
    p2 = Process(target=add100, args=(shared_number,))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    # Here the end value is not 200 most of the time. The reason is that race condition occurs;
    # That is, two processing are trying to access or modify the same shared variable at the same time
    print('number at the end is', shared_number.value)

Number at the beginning is 0
number at the end is 175


In [11]:
# We can use 'Lock' module to prevent what happened in the above cell
from multiprocessing import Process, Value, Array, Lock
import time

def add100(number, lock):
    for i in range(100):
        time.sleep(0.01)
        with lock:
            number.value += 1

if __name__ == "__main__":
    
    lock = Lock()
    shared_number = Value('i', 0) # two arguments, data type (i for integer) and starting value 
    print("Number at the beginning is", shared_number.value)
    
    p1 = Process(target=add100, args=(shared_number,lock))
    p2 = Process(target=add100, args=(shared_number,lock))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    # Here the end value is not 200 most of the time. The reason is that race condition occurs;
    # That is, two processing are trying to access or modify the same shared variable at the same time
    print('number at the end is', shared_number.value)

Number at the beginning is 0
number at the end is 200


In [14]:
# Now let's try to work on Array
# We can use 'Lock' module to prevent what happened in the above cell
from multiprocessing import Process, Value, Array, Lock
import time

def add100(numbers, lock):
    for i in range(100):
        time.sleep(0.01)
        for i in range(len(numbers)):
            with lock:
                numbers[i] += 1

if __name__ == "__main__":
    
    lock = Lock()
    shared_array = Array('d', [0.0, 100.0, 200.0]) # two arguments, data type (d for double) and starting value 
    print("Array at the beginning is", shared_array[:])
    
    p1 = Process(target=add100, args=(shared_array,lock))
    p2 = Process(target=add100, args=(shared_array,lock))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    # Here the end value is not 200 most of the time. The reason is that race condition occurs;
    # That is, two processing are trying to access or modify the same shared variable at the same time
    print("Array at the end is", shared_array[:])

Array at the beginning is [0.0, 100.0, 200.0]
Array at the end is [200.0, 300.0, 400.0]


In [19]:
# Queue
from multiprocessing import Queue

def square(numbers, queue):
    for i in numbers:
        queue.put(i*i)
    
def make_negative(numbers, queue):
    for i in numbers:
        queue.put(-1*i)

if __name__ == "__main__":
    
    numbers = range(1, 6)
    q = Queue()
    
    p1 = Process(target=square, args=(numbers, q))
    p2 = Process(target=make_negative, args=(numbers, q))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    while not q.empty():
        print(q.get())

1
4
9
16
25
-1
-2
-3
-4
-5


In [20]:
# Processing pool

from multiprocessing import Pool

def cube(number):
    return number * number * number

if __name__ == "__main__":
    
    numbers = range(10)
    pool = Pool()
    # four important methods to know with Pool: map, apply, join, close
    result = pool.map(cube, numbers)
    
    pool.close()
    pool.join()
    
    print(result)
    

[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]
